In [1]:
import numpy as np
import csv
import pandas as pd
from sklearn.model_selection import KFold
from sklearn.decomposition import PCA
from sklearn.linear_model import RidgeCV
from sklearn.metrics import mean_squared_error

def load_file(train_t0,train_t1,test_t0):
    with open(train_t0, 'r') as f:
        file = list(csv.reader(f, delimiter=','))
    with open(train_t1, 'r') as f:
        file_1 = list(csv.reader(f, delimiter=','))
    with open(test_t0, 'r') as f:
        file_2 = list(csv.reader(f, delimiter=','))
    D = np.delete(file, 0, 0)  
    D = D.astype(np.float)
    X = np.delete(D, 0, 1)  #X_TRAIN
    D = np.delete(file_2, 0, 0) 
    D = D.astype(np.float)
    X_TR = np.delete(D, 0, 1) #X_TEST
    E = np.delete(file_1, 0, 0) 
    E = E.astype(np.float)
    Y = np.delete(E, 0, 1) #Y_TRAIN
    X_train = X[0:150]  #TRAIN_T0
    X_test = X_TR[0:80]  #TEST_T0
    Y_train = Y[0:150]   #train_t1
    X_train,Y_train,X_test = zero_columns_drop(X_train,X_test,Y_train)
    return X_train,Y_train,X_test

def zero_columns_drop(X_train,X_test,Y_train):
    X_train = np.delete(X_train, 3, 1)
    X_train = np.delete(X_train, 464, 1)
    X_train = np.delete(X_train, 466, 1)
    X_train = np.delete(X_train, 493, 1)
    X_train = np.delete(X_train, 495, 1)
    X_train = np.delete(X_train, 522, 1)
    #####
    X_test = np.delete(X_test, 3, 1)
    X_test = np.delete(X_test, 464, 1)
    X_test = np.delete(X_test, 466, 1)
    X_test = np.delete(X_test, 493, 1)
    X_test = np.delete(X_test, 495, 1)
    X_test = np.delete(X_test, 522, 1)
    #####
    Y_train = np.delete(Y_train, 3, 1)
    Y_train = np.delete(Y_train, 464, 1)
    Y_train = np.delete(Y_train, 466, 1)
    Y_train = np.delete(Y_train, 493, 1)
    Y_train = np.delete(Y_train, 495, 1)
    Y_train = np.delete(Y_train, 522, 1)
    return X_train,Y_train,X_test

def train_model(model,X,Y):
    kf = KFold(n_splits=5,random_state=0)
    arr = []
    for train_indices, test_indices in kf.split(X):
        X_train, y_train = X[train_indices], Y[train_indices] #xe karsılık gelen datalar eşlendi
        X_test, y_test = X[test_indices], Y[test_indices]
        pca = PCA(n_components=120)
        X_train = pca.fit_transform(X_train)
        y_train = pca.transform(y_train)
        X_test = pca.transform(X_test)
        model.fit(X_train,y_train)
        predicted_Y_test= model.predict(X_test)
        predicted_Y_test = pca.inverse_transform(predicted_Y_test)
        arr.append(mean_squared_error(y_test, predicted_Y_test))
    return min(arr)

def predict_model(model,X,Y,X_test):
    pca = PCA(n_components=120) #120 feature
    X_train = pca.fit_transform(X)
    y_train = pca.transform(Y)
    X_test = pca.transform(X_test)
    model.fit(X_train,y_train)
    predicted_Y_test= model.predict(X_test)
    predicted_Y_test = pca.inverse_transform(predicted_Y_test)
    n,m = X_test.shape #n for dimension
    X0 = np.zeros((n,1))
    predicted_Y_test = np.hstack((predicted_Y_test[:,:3], X0, predicted_Y_test[:,3:]))
    predicted_Y_test = np.hstack((predicted_Y_test[:,:465], X0, predicted_Y_test[:,465:]))
    predicted_Y_test = np.hstack((predicted_Y_test[:,:468], X0, predicted_Y_test[:,468:]))
    predicted_Y_test = np.hstack((predicted_Y_test[:,:496], X0, predicted_Y_test[:,496:]))
    predicted_Y_test = np.hstack((predicted_Y_test[:,:499], X0, predicted_Y_test[:,499:]))
    predicted_Y_test = np.hstack((predicted_Y_test[:,:527], X0, predicted_Y_test[:,527:]))
    write_output(predicted_Y_test.flatten())
    print("Submission file created! ")
    
def write_output(predictions):

    submissionFile = [["ID", "Predicted"]]
    for i, prediction in enumerate(predictions):
        submissionFile.append([i, prediction])

    # Write to file;
    with open('Submission.csv', 'w') as csvFile:
        writer = csv.writer(csvFile)
        writer.writerows(submissionFile)

X_train,Y_train,X_test = load_file('train_t0.csv','train_t1.csv','test_t0.csv')
print("Train RidgeCV model MSE result: ", train_model(RidgeCV(),X_train, Y_train))
predict_model(RidgeCV(),X_train, Y_train,X_test)

Train RidgeCV model MSE result:  0.002329664770499526
Submission file created! 
